# Step 1: Import libraries

In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as stats

# Plotly setup for notebooks
pio.renderers.default = "notebook"


# Step 2: Load and preprocess dataset

In [ ]:
# Step 2: Load and preprocess dataset

import pandas as pd

FILE_PATH = "https://raw.githubusercontent.com/tvgerwe/ENEXIS/refs/heads/main/workspaces/sharell/df_filtered.csv"
print(f"📂 Loading: {FILE_PATH}")

# Load and prepare
df = pd.read_csv(FILE_PATH, index_col=0, parse_dates=True)
df.index = df.index.tz_localize(None)  # Remove timezone if present
df = df.asfreq("h").ffill()

# Select only necessary columns
required_cols = ["Price", "Load", "production_fossilhardcoal"]
missing = [col for col in required_cols if col not in df.columns]
if missing:
    raise ValueError(f"❌ Missing required columns: {missing}")

df = df[required_cols]
df["Price"] = df["Price"] / 1000  # Convert to EUR/kWh

# Preview
print(f"✅ Data shape: {df.shape}")
print(f"📅 Date range: {df.index.min()} → {df.index.max()}")
print(df.head())


# Step 3: Train-Test Split

In [ ]:
# Step 3: Train-Test Split

train_start = "2025-02-24"
train_end   = "2025-03-24 00:00:00"
test_start  = "2025-03-24 00:00:01"
test_end    = "2025-03-31"

train = df.loc[train_start:train_end]
test  = df.loc[test_start:test_end]

# Simple leakage check
assert train.index.max() < test.index.min(), "❌ Data leakage: train ends after test begins!"

# Overview
print(f"✅ Train: {train.index.min()} → {train.index.max()} | {len(train)} records")
print(f"✅ Test : {test.index.min()} → {test.index.max()} | {len(test)} records")


# Step 4: Feature Engineering

In [ ]:
# Step 4: Feature Engineering

from sklearn.preprocessing import StandardScaler

# Define target and features
target_col = "Price"
feature_cols = ["Load", "production_fossilhardcoal"]

# Extract features and target
X_train, y_train = train[feature_cols], train[target_col]
X_test, y_test   = test[feature_cols], test[target_col]

# Ensure index alignment
X_train = X_train.loc[y_train.index]
X_test = X_test.loc[y_test.index]

# Scale features (important for SARIMAX stability)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled  = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

# R-values (pre-scaling, on original features)
print("📈 R-values (correlation with Price):")
r_values = X_train.corrwith(y_train).sort_values(ascending=False)
for feature, r in r_values.items():
    print(f"  {feature:30s} → R = {r:.4f}")

# Shape check
print(f"\n✅ X_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"✅ X_test : {X_test.shape}  | y_test : {y_test.shape}")


# Step 5: Model Training

In [ ]:
# Step 5: Model Training (manual vs auto_arima, evaluated on RMSE)

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pmdarima import auto_arima
import numpy as np
import warnings

warnings.filterwarnings("ignore")  # Clean output (e.g., force_all_finite warnings)

def train_and_forecast(y_train, y_test, order, seasonal_order):
    model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order,
                    enforce_stationarity=False, enforce_invertibility=False)
    result = model.fit(disp=False)
    forecast = result.forecast(steps=len(y_test))
    rmse = np.sqrt(mean_squared_error(y_test, forecast))
    return rmse, forecast

# Your manual config
manual_order = (1, 1, 3)
manual_seasonal = (0, 1, 1, 24)

print("🧪 Testing manual SARIMA config...")
manual_rmse, manual_forecast = train_and_forecast(y_train, y_test, manual_order, manual_seasonal)
print(f"📉 Manual RMSE: {manual_rmse:.6f} EUR/kWh")

# AutoARIMA with lighter config (reduced search space)
print("\n⚙️ Running light auto_arima (may take ~1–2 mins)...")

auto_model = auto_arima(
    y_train,
    seasonal=True,
    m=24,
    max_order=10,
    max_p=2, max_q=2,
    max_P=1, max_Q=1,
    max_d=1, max_D=1,
    stepwise=True,
    suppress_warnings=True,
    error_action="ignore",
    trace=False
)

auto_order = auto_model.order
auto_seasonal = auto_model.seasonal_order

print(f"✅ auto_arima suggested: Order={auto_order}, Seasonal={auto_seasonal}")

# Forecast and evaluate RMSE
print("🧪 Testing auto_arima SARIMA config...")
auto_rmse, auto_forecast = train_and_forecast(y_train, y_test, auto_order, auto_seasonal)
print(f"📉 AutoARIMA RMSE: {auto_rmse:.6f} EUR/kWh")

# Compare and select
if auto_rmse < manual_rmse:
    print("🚀 Using auto_arima config (lower RMSE).")
    best_order = auto_order
    best_seasonal_order = auto_seasonal
    final_forecast = auto_forecast
    final_rmse = auto_rmse
else:
    print("🔒 Using manual config (lower RMSE).")
    best_order = manual_order
    best_seasonal_order = manual_seasonal
    final_forecast = manual_forecast
    final_rmse = manual_rmse

print(f"\n✅ Final config: Order={best_order}, Seasonal={best_seasonal_order}")
print(f"🏁 Final RMSE: {final_rmse:.6f} EUR/kWh")


# Step 6: Forecasting (SARIMA Univariate)

In [ ]:
# Step 6: Forecasting (SARIMA Univariate)

import plotly.graph_objects as go
import pandas as pd

# Align forecast index with test set
forecast_index = y_test.index
forecast_series = pd.Series(final_forecast, index=forecast_index)

# Plot actual vs forecast
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=forecast_index,
    y=y_test,
    mode='lines',
    name='Actual Price',
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=forecast_index,
    y=forecast_series,
    mode='lines',
    name='Forecast (SARIMA)',
    line=dict(color='blue', dash='dash')
))

fig.update_layout(
    title="SARIMA Forecast vs Actual (Univariate)",
    xaxis_title="Date",
    yaxis_title="Price (EUR/kWh)",
    template="plotly_white",
    hovermode="x unified"
)

fig.show()


# Step 6b: Naive Forecast

In [ ]:
# Step 6b: Naive Forecast

# Shift training series by 24 hours to match forecast horizon
naive_forecast = y_test.copy()
seasonal_period = 24

# Create a naive forecast by repeating the last 24h of training
repeats = int(np.ceil(len(y_test) / seasonal_period))
naive_values = np.tile(y_train[-seasonal_period:].values, repeats)[:len(y_test)]
naive_forecast[:] = naive_values

# Calculate RMSE
naive_rmse = np.sqrt(mean_squared_error(y_test, naive_forecast))
print(f"Naive RMSE: {naive_rmse:.6f} EUR/kWh")


# Step 6c: Benchmark Forecast (Oxygent)

In [ ]:
# Step 6c: Benchmark Forecast (Oxygent)

from sklearn.metrics import mean_squared_error

test_start = pd.Timestamp("2025-03-24 00:00:01")
test_end   = pd.Timestamp("2025-03-31 23:00:00")

# Load and process Oxygent benchmark predictions
oxygent_path = "https://raw.githubusercontent.com/tvgerwe/ENEXIS/refs/heads/main/src/data/price_predictions_oxygent/Price_Preds_Processed_20250407.csv"
oxy_df = pd.read_csv(oxygent_path)

oxy_df["target_hour"] = pd.to_datetime(oxy_df["x"] * 100000, unit="ms", utc=True).dt.tz_localize(None).dt.floor("H")
oxy_df = oxy_df[(oxy_df["target_hour"] >= test_start) & (oxy_df["target_hour"] <= test_end)]

# Get actuals from your main df
actuals_df = df[["Price"]].copy()
actuals_df.index.name = "target_hour"
actuals_df = actuals_df.reset_index()

# Merge
merged_oxy = pd.merge(oxy_df, actuals_df, on="target_hour", how="inner")

# Rename merged columns for clarity
merged_oxy.rename(columns={"Price_y": "Price"}, inplace=True)

# Now drop missing
merged_oxy = merged_oxy.dropna(subset=["y", "Price"])

# Evaluate RMSE
oxy_rmse = np.sqrt(mean_squared_error(merged_oxy["Price"], merged_oxy["y"]))
print(f"Oxygent Benchmark RMSE: {oxy_rmse:.6f} EUR/kWh")


#step 7: Plotting All Models

In [ ]:
#step 7: Plotting All Models
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# === Refit SARIMAX model (if needed) ===
sarimax_model = SARIMAX(
    y_train,
    exog=X_train_scaled,
    order=best_order,
    seasonal_order=best_seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarimax_result = sarimax_model.fit(disp=False)
sarimax_forecast = sarimax_result.forecast(steps=len(y_test), exog=X_test_scaled)
sarimax_series = pd.Series(sarimax_forecast, index=y_test.index)

# === Prepare Oxygent series ===
oxy_latest = merged_oxy.sort_values("timestamp").drop_duplicates(subset="target_hour", keep="last")
oxy_latest = oxy_latest.set_index("target_hour").sort_index()
oxygent_series = oxy_latest.reindex(y_test.index)["y"]

# === Plot all models ===
fig = go.Figure()

fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='Actual', line=dict(color='black')))
fig.add_trace(go.Scatter(x=forecast_series.index, y=forecast_series, mode='lines', name='SARIMA Forecast', line=dict(color='blue', dash='dot')))
fig.add_trace(go.Scatter(x=sarimax_series.index, y=sarimax_series, mode='lines', name='SARIMAX Forecast', line=dict(color='green', dash='dash')))
fig.add_trace(go.Scatter(x=naive_forecast.index, y=naive_forecast, mode='lines', name='Naive Forecast', line=dict(color='red', dash='dot')))
fig.add_trace(go.Scatter(x=oxygent_series.index, y=oxygent_series, mode='lines', name='Oxygent Benchmark', line=dict(color='orange', dash='dash')))

# === Vertical gridlines at midnight + date annotations ===
daily_lines = y_test.index.normalize().unique()

vertical_day_lines = [
    dict(
        type='line',
        x0=day,
        x1=day,
        y0=0,
        y1=1,
        xref='x',
        yref='paper',
        line=dict(color='lightgray', width=1, dash='dot')
    )
    for day in daily_lines
]

date_annotations = [
    dict(
        x=day,
        y=1.02,
        xref='x',
        yref='paper',
        text=day.strftime('%d-%b'),
        showarrow=False,
        font=dict(size=12, color='black'),
        align="center"
    )
    for day in daily_lines
]

# === Final layout ===
fig.update_layout(
    title="📊 Forecast Comparison: All Models",
    xaxis_title="Time",
    yaxis_title="Price (EUR/kWh)",
    template="plotly_white",
    hovermode="x unified",
    width=1500,
    height=600,
    xaxis=dict(
        tickformat="%H:%M",
        dtick=7200000  # 2-hour ticks
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    shapes=vertical_day_lines,
    annotations=date_annotations
)

fig.show()


In [ ]:
# Step 8: Daily RMSE Summary Table

In [ ]:
# Step 8: Daily RMSE Summary Table

from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import plotly.express as px

# Combine all forecasts into a single DataFrame
rmse_df = pd.DataFrame({
    "Actual": y_test,
    "SARIMA": forecast_series,
    "SARIMAX": sarimax_series,
    "Naive": naive_forecast,
    "Oxygent": oxygent_series
}).dropna()

# Add date column for grouping
rmse_df["date"] = rmse_df.index.date

# Group by date and calculate RMSE per model
daily_rmse = rmse_df.groupby("date").apply(
    lambda group: pd.Series({
        "SARIMA_RMSE": np.sqrt(mean_squared_error(group["Actual"], group["SARIMA"])),
        "SARIMAX_RMSE": np.sqrt(mean_squared_error(group["Actual"], group["SARIMAX"])),
        "Naive_RMSE": np.sqrt(mean_squared_error(group["Actual"], group["Naive"])),
        "Oxygent_RMSE": np.sqrt(mean_squared_error(group["Actual"], group["Oxygent"]))
    })
).reset_index()
# Add winner column based on min RMSE
rmse_cols = ["SARIMA_RMSE", "SARIMAX_RMSE", "Naive_RMSE", "Oxygent_RMSE"]
daily_rmse["Winner"] = daily_rmse[rmse_cols].idxmin(axis=1).str.replace("_RMSE", "")

# Sort and round
daily_rmse.sort_values("date", inplace=True)
daily_rmse = daily_rmse.round(6)

# Show summary table
from IPython.display import display
display(daily_rmse)

# Optional: bar plot
fig = px.bar(
    daily_rmse.melt(id_vars=["date", "Winner"], var_name="Model", value_name="RMSE"),
    x="date",
    y="RMSE",
    color="Model",
    barmode="group",
    title="📊 Daily RMSE Comparison by Model"
)
fig.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error
from IPython.display import display

# === Collect forecast errors by day ahead ===
forecast_horizon_errors = {
    "Model": [],
    "Horizon (days)": [],
    "RMSE": []
}

# Assume predictions are already aligned: y_test, forecast_series, sarimax_series, naive_forecast, oxygent_series
forecast_df = pd.DataFrame({
    "Actual": y_test,
    "SARIMA": forecast_series,
    "SARIMAX": sarimax_series,
    "Naive": naive_forecast,
    "Oxygent": oxygent_series
}).dropna()

# Add hour index as time delta from forecast start
forecast_df["hour"] = (forecast_df.index - forecast_df.index[0]).total_seconds() / 3600
forecast_df["day"] = (forecast_df["hour"] // 24).astype(int) + 1  # Day 1 to 7

# === Calculate RMSE per model per forecast day ===
for day in range(1, 8):
    day_data = forecast_df[forecast_df["day"] == day]
    for model in ["SARIMA", "SARIMAX", "Naive", "Oxygent"]:
        rmse = np.sqrt(mean_squared_error(day_data["Actual"], day_data[model]))
        forecast_horizon_errors["Model"].append(model)
        forecast_horizon_errors["Horizon (days)"].append(day)
        forecast_horizon_errors["RMSE"].append(rmse)

rmse_horizon_df = pd.DataFrame(forecast_horizon_errors)

# === Overall RMSE per model ===
overall_rmse = forecast_df[["SARIMA", "SARIMAX", "Naive", "Oxygent"]].apply(
    lambda preds: np.sqrt(mean_squared_error(forecast_df["Actual"], preds))
).reset_index()
overall_rmse.columns = ["Model", "Overall RMSE (EUR/kWh)"]

# === Best model per forecast day ===
pivot = rmse_horizon_df.pivot(index="Horizon (days)", columns="Model", values="RMSE")
pivot["Best Model"] = pivot.idxmin(axis=1)

# === Print tables ===
print("📊 Overall RMSE per Model:")
display(overall_rmse.round(6))

print("\n📈 RMSE per Forecast Horizon:")
display(rmse_horizon_df.pivot(index="Horizon (days)", columns="Model", values="RMSE").round(6))

print("\n🏆 Best Model per Day Ahead:")
display(pivot[["Best Model"]])

# === Plot RMSE per forecast horizon ===
fig = px.line(
    rmse_horizon_df,
    x="Horizon (days)",
    y="RMSE",
    color="Model",
    title="📉 RMSE per Forecast Day Ahead",
    markers=True,
    template="plotly_white"
)
fig.update_layout(yaxis_title="RMSE (EUR/kWh)", xaxis=dict(dtick=1))
fig.show()

# === Optional: export to CSV ===
# rmse_horizon_df.to_csv("forecast_horizon_rmse.csv", index=False)
# overall_rmse.to_csv("overall_rmse_summary.csv", index=False)


In [ ]:
# === 4. RMSE per dag per methode ===
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display

strategies = ["SARIMAX", "Lag_1", "Recursive", "Combined"]
rmse_results = []

# ✅ Dagelijks RMSE per model
for day, group in eval_df.groupby("date"):
    row = {"date": day}
    for strategy in strategies:
        row[f"{strategy}_RMSE"] = np.sqrt(mean_squared_error(group["actual"], group[strategy]))
    rmse_results.append(row)

# ✅ DataFrame aanmaken
rmse_df = pd.DataFrame(rmse_results)
rmse_df = rmse_df.round(6).sort_values("date")

# ✅ Gemiddelde per modelstrategie
mean_row = {col: rmse_df[col].mean() for col in rmse_df.columns if col != "date"}
mean_row["date"] = "Gemiddelde"
rmse_df = pd.concat([rmse_df, pd.DataFrame([mean_row])], ignore_index=True)

# ✅ Tabel tonen
display(rmse_df)

# ✅ Visualisatie
fig = px.line(
    rmse_df.melt(id_vars=["date"], var_name="Model", value_name="RMSE"),
    x="date",
    y="RMSE",
    color="Model",
    title="📉 RMSE per Dag per Strategie (inclusief Gemiddelde)"
)
fig.update_layout(xaxis=dict(type="category"))
fig.show()


# attempting over here

In [ ]:
# 1. Bepaal best presterend model obv RMSE
rmse_by_model = forecast_df[['SARIMA', 'SARIMAX', 'Naive', 'Oxygent']].apply(
    lambda col: ((forecast_df['Actual'] - col) ** 2).mean() ** 0.5
)
best_model = rmse_by_model.idxmin()
print(f"✔️ Using best model based on RMSE: {best_model}")

# 2. Laad device-profielen
device_profiles = pd.read_json(
    "https://raw.githubusercontent.com/tvgerwe/ENEXIS/refs/heads/main/src/data/profiles/device_usage_profile"
).to_dict(orient='records')

# 3. Bereken optimale verbruiksuren
optimal_schedules = []

for device in device_profiles:
    name = device['Device']
    duration = device['Duration_h']
    uses = device['Weekly_Uses']
    total_hours_needed = duration * uses
    allowed_hours = device['Constraints']['Allowed_Hours']
    min_block = device['Constraints']['Min_Block_Hours']  # voor later gebruik

    # Filter forecast op toegestane uren
    valid_hours = forecast_df[forecast_df['hour'].isin(allowed_hours)].copy()

    # Sorteer op voorspelde prijs van best presterend model
    valid_hours_sorted = valid_hours.sort_values(by=best_model)

    selected_hours = []
    total_hours_allocated = 0

    for _, row in valid_hours_sorted.iterrows():
        if total_hours_allocated >= total_hours_needed:
            break

        selected_hours.append({
            "Device": name,
            "Day": row['day'],
            "Hour": row['hour'],
            "Predicted_Price": row[best_model]
        })
        total_hours_allocated += 1

    optimal_schedules.extend(selected_hours)

# 4. Maak een overzicht van geselecteerde uren
df_schedule = pd.DataFrame(optimal_schedules)

# 5. Rapport per apparaat
df_report = df_schedule.groupby('Device').agg(
    Total_Hours=('Hour', 'count'),
    Avg_kWh_Price=('Predicted_Price', 'mean'),
    Total_kWh_Cost=('Predicted_Price', 'sum')
).reset_index()

# 6. Bekijk resultaten
display(df_schedule.head(10))
display(df_report)
